---
* *** 원본 소스는 홍콩과기대 김성훈 교수님의  [Github link](https://github.com/hunkim/DeepLearningZeroToAll)를 참조하세요 ***     
* *** Watson studio notebook 에서 작업 가능하도록 수정하였으며***
* *** 한글로 설명된 부분은 제가 스터디 하면서 이해한 내용을 추가한 것입니다. ***
---

# Lab 6 Softmax Classifier
* ***Lab 6-1하고 동일하지만, zoo data file을 input으로 씁니다.***
* ***input file 앞부분에 comment 처리된 데이터가 들어 있어서, np.loadtxt 와  pd.read_csv 간의 default 값이 달라서 처리가 필요합니다***
* ***앞에서는 one hot encoding 개념을 수작업으로 처리했다면, 여기서는 함수(one_hot, reshape)로 처리합니다.***

In [7]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # for reproducibility

* ***insert to code 로 불러온 코드***

In [8]:
import sys
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_ace6f71a1b9946cbb684ca0d9e6c34c0 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='xBNCWwO4mooXZRHuqZ71nRmyIlQt8ca3ZWN8pOo56X69Dx',     ##masking, use your own
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_ace6f71a1b9946cbb684ca0d9e6c34c0.get_object(Bucket='tensorflowlabwithwatsonstudio-donotdelete-pr-neiwaip4a29fcg',Key='data-04-zoo.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body,comment='#')
xy = df_data_1.values
df_data_1.head()

,1,0,0.1,1.1,0.2,0.3,1.2,1.3,1.4,1.5,0.4,0.5,4,0.6,0.7,1.6,0.8
0,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,0
1,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,3
2,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,0
3,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,0
4,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,0


* ***pd.read_csv(body,comment='#')   --> default comment = none***
* ***np.loadtxt(aa,comment='#') --> default comment = '#'***

In [9]:
# Predicting animal type based on various features
# xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape)

nb_classes = 7  # 0 ~ 6

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])  # 0 ~ 6

(100, 16) (100, 1)


In [10]:
y_data[0:10,:]

array([[0],
       [3],
       [0],
       [0],
       [0],
       [0],
       [3],
       [3],
       [0],
       [0]])

In [11]:
Y_one_hot = tf.one_hot(Y, nb_classes)  # one hot
print("one_hot", Y_one_hot)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
print("reshape", Y_one_hot)

W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

one_hot Tensor("one_hot:0", shape=(?, 1, 7), dtype=float32)
reshape Tensor("Reshape:0", shape=(?, 7), dtype=float32)


In [12]:
# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

In [16]:
# Cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [17]:
prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [18]:
# Launch graph
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(2000):
    sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
    if step % 100 == 0:
        loss, acc = sess.run([cost, accuracy], feed_dict={
                             X: x_data, Y: y_data})
        print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(
            step, loss, acc))

Step:     0	Loss: 5.153	Acc: 37.00%
Step:   100	Loss: 0.802	Acc: 80.00%
Step:   200	Loss: 0.488	Acc: 88.00%
Step:   300	Loss: 0.350	Acc: 90.00%
Step:   400	Loss: 0.272	Acc: 94.00%
Step:   500	Loss: 0.222	Acc: 95.00%
Step:   600	Loss: 0.187	Acc: 97.00%
Step:   700	Loss: 0.161	Acc: 97.00%
Step:   800	Loss: 0.141	Acc: 97.00%
Step:   900	Loss: 0.124	Acc: 97.00%
Step:  1000	Loss: 0.111	Acc: 97.00%
Step:  1100	Loss: 0.101	Acc: 99.00%
Step:  1200	Loss: 0.092	Acc: 100.00%
Step:  1300	Loss: 0.084	Acc: 100.00%
Step:  1400	Loss: 0.078	Acc: 100.00%
Step:  1500	Loss: 0.072	Acc: 100.00%
Step:  1600	Loss: 0.068	Acc: 100.00%
Step:  1700	Loss: 0.064	Acc: 100.00%
Step:  1800	Loss: 0.060	Acc: 100.00%
Step:  1900	Loss: 0.057	Acc: 100.00%


In [19]:
# Let's see if we can predict
pred = sess.run(prediction, feed_dict={X: x_data})
# y_data: (N,1) = flatten => (N, ) matches pred.shape
for p, y in zip(pred, y_data.flatten()):
    print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 1 True Y: 1
[True] Prediction: 3 True Y: 3
[True] Prediction: 6 True Y: 6
[True] Prediction: 6 True Y: 6
[True] Prediction: 6 True Y: 6
[True] Prediction: 1 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 1 True Y: 1
[True] Prediction: 1 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 1 True Y: 1
[True] Prediction: 5 True Y: 5
[True] Prediction: 4 True Y: 4
[True] Prediction: 4 True Y: 4
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 5 True Y: 5
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] P

In [ ]:
'''
Step:     0 Loss: 5.106 Acc: 37.62%
Step:   100 Loss: 0.800 Acc: 79.21%
Step:   200 Loss: 0.486 Acc: 88.12%
Step:   300 Loss: 0.349 Acc: 90.10%
Step:   400 Loss: 0.272 Acc: 94.06%
Step:   500 Loss: 0.222 Acc: 95.05%
Step:   600 Loss: 0.187 Acc: 97.03%
Step:   700 Loss: 0.161 Acc: 97.03%
Step:   800 Loss: 0.140 Acc: 97.03%
Step:   900 Loss: 0.124 Acc: 97.03%
Step:  1000 Loss: 0.111 Acc: 97.03%
Step:  1100 Loss: 0.101 Acc: 99.01%
Step:  1200 Loss: 0.092 Acc: 100.00%
Step:  1300 Loss: 0.084 Acc: 100.00%
...
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
'''